In [27]:
import pandas as pd
import openpyxl

## Converting csv into pandas dataframe

Converting the **MERGEDXXXX_YY_PP.csv** files to dataframes with `read_csv()` causes an error regarding mixed types in columns. I'm using the `low_memory=False` argument to continue, but should consider using the `converters` or `dtype` args.

With `converters`:    
```Python
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)   
    except:        
        return ''

pd.read_csv('file.csv',converters={'first_column': convert_dtype,'second_column': convert_dtype})
```

With `dtype`:
```Python
pd.read_csv('file.csv', dtype={'first_column'='string', 'second_column'='string'})
```

The problem with either of these approaches in this context is that this dataset is wide (2000+ columns) and a lot of columns seem to have mixed columns. I will research how to solve this problem without ignoring memory concerns.

The advantage we have when using `low_memory=False` is that the dataset is not large (6000+ rows), so the converstion from csv to dataframe is quick (~80ms on my machine)

Reference: <a href="https://www.roelpeters.be/solved-dtypewarning-columns-have-mixed-types-specify-dtype-option-on-import-or-set-low-memory-in-pandas/">dtype warning: columns have mixed types</a>

In [3]:
df18 = pd.read_csv("data/MERGED2018_19_PP.csv",low_memory=False)
dfc18 = df18.copy() # create a copy of original df

## Analysis

First we want to drop all fully empty columns

In [4]:
dfc18 = dfc18.dropna(axis='columns',how='all')
print(f'original \'18 shape: {df18.shape} \n      after dropna: {dfc18.shape}')

original '18 shape: (6806, 2044) 
      after dropna: (6806, 711)


We removed 1333 fully empty columns. Note: *We should probably inspect what columns they are in case the null values provide some sort of insight*

Next, let's filter which schools we are focused on (4 year institutions):

In [5]:
is_4year = dfc18['HIGHDEG'] >= 3
dfc18_4yr = dfc18[is_4year]
dfc18_4yr.shape

(2725, 711)

We've got 2725 4-year schools. Let's see how many columns have null values, and go from there.

In [6]:
len(dfc18_4yr.columns[dfc18.isnull().any() == True].tolist())

693

In [7]:
rec_co = pd.read_csv("data/Most-Recent-Cohorts-All-Data-Elements.csv", low_memory=False)

In [8]:
rec_co.shape

(6806, 2045)

## Basic EDA Assignment

Extracting the dataset to be used for Milestone 1 of the project

In [22]:
# using dfc18_4yr from In[5]
# 'SCH_DEG' 'LOCALE2' 'UG' 'NPT4_PROG' 'NPT4_OTHER' 'NPT41_PROG' 'NPT42_PROG'
# 'NPT43_PROG' 'NPT44_PROG' 'NPT45_PROG' 'NPT41_OTHER' 'NPT42_OTHER' 'NPT43_OTHER'
# 'NPT44_OTHER' 'NPT45_OTHER' 'NPT4_048_PROG' 'NPT4_048_OTHER' 'NPT4_3075_PROG'
# 'NPT4_3075_OTHER' 'NPT4_75UP_PROG' 'NPT4_75UP_OTHER' 'NUM4_PROG' 'NUM4_OTHER' 
# 'NUM41_PROG' 'NUM42_PROG' 'NUM43_PROG' 'NUM44_PROG' 'NUM45_PROG' 'NUM41_OTHER' 
# 'NUM42_OTHER' 'NUM43_OTHER' 'NUM44_OTHER' 'NUM45_OTHER' 'UG25ABV' 'RPY_1YR_RT' 
# 'COMPL_RPY_1YR_RT' 'LO_INC_RPY_1YR_RT' 'MD_INC_RPY_1YR_RT' 'HI_INC_RPY_1YR_RT' 
# 'DEP_RPY_1YR_RT' 'IND_RPY_1YR_RT' 'PELL_RPY_1YR_RT' 'NOPELL_RPY_1YR_RT' 
# 'FEMALE_RPY_1YR_RT' 'MALE_RPY_1YR_RT' 'FIRSTGEN_RPY_1YR_RT' 'NOTFIRSTGEN_RPY_1YR_RT' 
# 'INC_PCT_LO' 'DEP_STAT_PCT_IND' 'IND_INC_PCT_LO' 'DEP_INC_PCT_LO' 'PAR_ED_PCT_1STGEN' 
# 'INC_PCT_M1' 'INC_PCT_M2' 'INC_PCT_H1' 'INC_PCT_H2'
m1_list = ['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'INSTURL',
           'NPCURL', 'MAIN', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL',
           'ST_FIPS', 'REGION', 'LOCALE', 'CCUGPROF', 'CCSIZSET', 'MENONLY',
           'WOMENONLY', 'RELAFFIL', 'ADM_RATE', 'DISTANCEONLY', 'UGDS', 'CURROPER',
           'NPT4_PUB', 'NPT4_PRIV', 'NPT41_PUB', 'NPT42_PUB', 'NPT43_PUB', 'NPT44_PUB',
           'NPT45_PUB', 'NPT41_PRIV', 'NPT42_PRIV', 'NPT43_PRIV', 'NPT44_PRIV',
           'NPT45_PRIV', 'NPT4_048_PUB', 'NPT4_048_PRIV', 'NPT4_3075_PUB', 
           'NPT4_3075_PRIV', 'NPT4_75UP_PUB', 'NPT4_75UP_PRIV', 'NUM4_PUB', 'NUM4_PRIV',
           'NUM41_PUB', 'NUM42_PUB', 'NUM43_PUB', 'NUM44_PUB', 'NUM45_PUB', 'NUM41_PRIV',
           'NUM42_PRIV', 'NUM43_PRIV', 'NUM44_PRIV', 'NUM45_PRIV', 'COSTT4_A', 'COSTT4_P', 
           'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'TUITIONFEE_PROG', 'PFTFAC', 'PCTPELL', 'RET_FT4',
           'RET_PT4', 'PCTFLOAN']

In [23]:
df_m1 = dfc18_4yr[m1_list]

In [24]:
df_m1.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,INSTURL,NPCURL,MAIN,...,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,PFTFAC,PCTPELL,RET_FT4,RET_PT4,PCTFLOAN
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,1,...,22489.0,NaN,9744.0,18354.0,NaN,0.7411,0.7067,0.6087,1.0000,0.7503
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,https://www.uab.edu,https://uab.studentaidcalculator.com/survey.aspx,1,...,24347.0,NaN,8568.0,19704.0,NaN,0.7766,0.3632,0.8186,0.4648,0.5127
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,www.amridgeuniversity.edu,www2.amridgeuniversity.edu:9091/,1,...,17680.0,NaN,6900.0,6900.0,NaN,1.0000,0.7673,NaN,NaN,0.8962
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,www.uah.edu,finaid.uah.edu/,1,...,23441.0,NaN,10714.0,22362.0,NaN,0.6544,0.2698,0.8288,0.0909,0.4192
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,www.alasu.edu,www.alasu.edu/cost-aid/tuition-costs/net-price...,1,...,21476.0,NaN,11068.0,19396.0,NaN,0.5826,0.7448,0.5868,0.1250,0.7845


In [28]:
df_m1.to_excel('data/milestone1_prelim_dataset.xlsx', index = False)